# Perceptrón para Análisis de Sentimiento en Español

## Objetivo

En esta actividad vas a implementar desde cero un modelo de Perceptrón en Python usando solo NumPy. Vamos a entrenarlo con frases breves escritas en español rioplatense para que aprenda a reconocer si una frase tiene un sentimiento positivo o negativo.

El objetivo es entender cómo funciona una neurona artificial básica, cómo se ajustan sus pesos durante el aprendizaje, y cómo puede hacer predicciones en base a un conjunto pequeño de frases.

### ¿Qué es un perceptrón?

El perceptrón es el modelo más simple de neurona artificial. Fue propuesto en 1958 por Frank Rosenblatt y representa la base fundamental de las redes neuronales modernas.

**Funcionamiento básico:**
1. Recibe múltiples entradas (features)
2. Multiplica cada entrada por un peso
3. Suma todos los resultados y agrega un sesgo (bias)
4. Aplica una función de activación para decidir la salida

Es un modelo lineal que solo puede aprender patrones que sean linealmente separables.

## 1. Datos de entrenamiento

Vamos a usar un pequeño conjunto de frases etiquetadas como positivas (1) o negativas (0). Las frases son simples como las que podríamos encontrar en una conversación cotidiana en Argentina.

También definimos un vocabulario básico de palabras clave que aparecen con frecuencia y que nos pueden ayudar a inferir el sentimiento.

In [ ]:
import numpy as np

# Frases con su etiqueta de sentimiento (1 = positivo, 0 = negativo)
frases = [
    "Amo el verano en Buenos Aires",
    "No me gusta el tráfico matutino",
    "Este asado está espectacular",
    "Qué bajón, perdí el colectivo",
    "Me encanta salir los domingos",
    "Detesto el calor húmedo"
]

etiquetas = np.array([1, 0, 1, 0, 1, 0])  # Etiquetas correspondientes

# Vocabulario manual con palabras claves de carga emocional
vocabulario = ["amo", "no", "gusta", "asado", "espectacular", "bajón", "perdí", "detesto", "calor", "húmedo"]

print(f"Total de frases: {len(frases)}")
print(f"Vocabulario: {len(vocabulario)} palabras clave")
print(f"\nPrimera frase: '{frases[0]}' → Sentimiento: {'Positivo' if etiquetas[0] == 1 else 'Negativo'}")

## 2. Representación numérica: Vectorización de frases

Las redes neuronales no pueden procesar texto directamente. Necesitamos convertir cada frase en un vector numérico.

### Bag of Words (Bolsa de Palabras)

Vamos a usar una representación simple llamada **bag of words**: para cada frase, creamos un vector binario que indica si cada palabra del vocabulario aparece (1) o no (0) en la frase.

**Ejemplo:**
- Vocabulario: ["amo", "no", "gusta", "asado"]
- Frase: "Amo el asado"
- Vector: [1, 0, 0, 1] (tiene "amo" y "asado", no tiene "no" ni "gusta")

**Limitación importante:** Este método no captura el orden de las palabras ni el contexto. "No me gusta" y "me gusta" tendrían vectores muy similares.

In [ ]:
def vectorizar(frase, vocabulario):
    """
    Convierte una frase en un vector binario según el vocabulario.
    
    Args:
        frase: String con la frase a vectorizar
        vocabulario: Lista de palabras clave
    
    Returns:
        Array de numpy con 1s y 0s
    """
    tokens = frase.lower().split()
    return np.array([1 if palabra in tokens else 0 for palabra in vocabulario])

# Aplicamos la función a todas las frases
X = np.array([vectorizar(frase, vocabulario) for frase in frases])

print("Matriz de features (X):")
print(f"Forma: {X.shape} (6 frases × 10 features)\n")
print("Vectores generados:")
for i, (frase, vector) in enumerate(zip(frases, X)):
    print(f"Frase {i+1}: {vector} → Sentimiento: {'Positivo' if etiquetas[i] == 1 else 'Negativo'}")

## 3. Definición del modelo: el Perceptrón

Un perceptrón es una función matemática que:

1. **Multiplica** cada entrada por un peso (weight)
2. **Suma** los resultados y agrega un sesgo (bias)
3. **Aplica** una función de activación para decidir si "dispara" o no

### Fórmula matemática:

```
z = (x₁ × w₁) + (x₂ × w₂) + ... + (xₙ × wₙ) + bias
salida = función_activación(z)
```

### Función de activación escalón:

```
Si z > 0 → salida = 1 (positivo)
Si z ≤ 0 → salida = 0 (negativo)
```

Vamos a inicializar los pesos aleatoriamente y entrenar el modelo para que aprenda de los errores.

In [ ]:
###############################################################################
# CONFIGURACIÓN DEL PERCEPTRÓN: INICIALIZACIÓN DE PARÁMETROS
###############################################################################

# ¿Por qué fijamos la semilla aleatoria?
# np.random.seed(42) hace que los valores aleatorios sean reproducibles.
# Cada vez que ejecutás este notebook, obtenés los mismos pesos iniciales.
# Esto es fundamental para debugging y para que los resultados sean comparables.
np.random.seed(42)

# ¿Por qué inicializamos los pesos aleatoriamente?
# Si todos los pesos fueran iguales (ej: todos en 0), el modelo no podría aprender.
# Necesitamos romper la simetría: cada peso debe empezar con un valor distinto.
#
# np.random.randn() genera números de una distribución normal (Gaussiana):
# - Media = 0
# - Desviación estándar = 1
# - Valores típicos: entre -2 y +2 (68% entre -1 y +1)
#
# ¿Por qué esta distribución?
# - Valores pequeños evitan saturación en funciones de activación
# - Centrados en 0 para que el modelo no tenga sesgo inicial hacia positivo o negativo
# - Distribución normal es la más común en inicialización de pesos
pesos = np.random.randn(len(vocabulario))

# ¿Por qué el bias empieza en 0.0?
# El bias (o sesgo) es un término que se suma a la combinación lineal.
# Matemáticamente: z = w₁x₁ + w₂x₂ + ... + wₙxₙ + bias
#
# Empezamos en 0.0 porque:
# - No tenemos información previa sobre si el dataset es mayormente positivo o negativo
# - El bias se ajustará durante el entrenamiento según los datos
# - Valor neutro que no predispone al modelo
bias = 0.0

print("Pesos iniciales (aleatorios):")
for i, (palabra, peso) in enumerate(zip(vocabulario, pesos)):
    print(f"  w[{palabra}] = {peso:.3f}")
print(f"\nBias inicial: {bias}")

###############################################################################
# FUNCIÓN DE ACTIVACIÓN: ESCALÓN (STEP FUNCTION)
###############################################################################

def activar(suma):
    """
    Función de activación escalón (step function).
    
    ¿Qué hace?
    Convierte un número real cualquiera en una decisión binaria (0 o 1).
    
    ¿Por qué usamos esta función?
    El perceptrón clásico de Rosenblatt (1958) usaba esta función porque:
    - Es simple de implementar y entender
    - Produce una decisión clara: positivo (1) o negativo (0)
    - Modela una neurona biológica que "dispara" o "no dispara"
    
    Matemáticamente:
    f(z) = 1  si z > 0
    f(z) = 0  si z ≤ 0
    
    Limitación importante:
    Esta función NO es diferenciable en z=0, lo que impide usar
    técnicas modernas como backpropagation. Por eso las redes modernas
    usan funciones como ReLU o Sigmoid.
    
    Args:
        suma: Valor numérico de la combinación lineal z = w·x + b
    
    Returns:
        1 si suma > 0, 0 en caso contrario
    """
    return 1 if suma > 0 else 0

###############################################################################
# FUNCIÓN DE PREDICCIÓN: CÁLCULO DE LA SALIDA DEL PERCEPTRÓN
###############################################################################

def predecir(x):
    """
    Calcula la predicción del perceptrón para un vector de entrada.
    
    ¿Cómo funciona?
    1. Producto punto: np.dot(x, pesos) calcula x₁w₁ + x₂w₂ + ... + xₙwₙ
    2. Suma el bias: z = producto_punto + bias
    3. Aplica activación: salida = activar(z)
    
    Ejemplo numérico:
    Supongamos:
    - x = [1, 0, 1, 0, 0, 0, 0, 0, 0, 0]  (vector de una frase)
    - pesos = [0.5, -0.3, 0.2, ...]       (pesos aprendidos)
    - bias = 0.1
    
    Entonces:
    - suma = (1)(0.5) + (0)(-0.3) + (1)(0.2) + ... + 0.1
    - suma = 0.5 + 0 + 0.2 + ... + 0.1 = 0.8
    - activar(0.8) = 1  (porque 0.8 > 0)
    - Predicción: Positivo
    
    ¿Por qué usamos producto punto?
    Es la operación fundamental del álgebra lineal que permite:
    - Combinar múltiples features en un solo valor
    - Ponderar cada feature por su importancia (peso)
    - Implementación eficiente en NumPy (optimizada en C)
    
    Args:
        x: Vector de features (array de numpy con 1s y 0s)
    
    Returns:
        Predicción: 0 (negativo) o 1 (positivo)
    """
    suma = np.dot(x, pesos) + bias
    return activar(suma)

print("\nFunciones definidas: activar() y predecir()")

## 4. Entrenamiento del modelo

Ahora vamos a entrenar el perceptrón ajustando los pesos según los errores que comete.

### Regla de aprendizaje del perceptrón:

Para cada ejemplo:
1. Calcular la predicción
2. Calcular el error: `error = etiqueta_real - predicción`
3. Si hay error (≠ 0), ajustar los pesos:
   - `peso_nuevo = peso_viejo + (tasa_aprendizaje × error × entrada)`
   - `bias_nuevo = bias_viejo + (tasa_aprendizaje × error)`

### Parámetros de entrenamiento:

- **Tasa de aprendizaje**: Controla qué tan grande es cada ajuste (0.1 es un valor común)
- **Épocas**: Cantidad de veces que recorremos todo el dataset

El entrenamiento se detiene cuando el modelo clasifica correctamente todos los ejemplos o llega al máximo de épocas.

In [ ]:
###############################################################################
# CONFIGURACIÓN DEL ENTRENAMIENTO: HIPERPARÁMETROS
###############################################################################

# ¿Qué es la tasa de aprendizaje (learning rate)?
# Es el tamaño del "paso" que damos cuando ajustamos los pesos.
#
# Matemáticamente: peso_nuevo = peso_viejo + (tasa_aprendizaje × error × entrada)
#
# ¿Por qué 0.1?
# - Muy grande (ej: 1.0): El modelo puede "saltar" la solución y no converger
# - Muy pequeña (ej: 0.001): El aprendizaje es muy lento, necesita muchas épocas
# - 0.1 es un valor moderado que suele funcionar bien para problemas simples
#
# Analogía: Si estás bajando una montaña en la niebla:
# - Tasa grande = pasos largos (rápido pero podés pasar la base)
# - Tasa pequeña = pasos cortos (lento pero seguro)
tasa_aprendizaje = 0.1

# ¿Qué es una época?
# Una época es un recorrido completo por todos los ejemplos de entrenamiento.
# En este caso, 1 época = procesar las 6 frases una vez.
#
# ¿Por qué 20 épocas?
# - Es un número arbitrario pero suficiente para este dataset pequeño
# - El perceptrón puede converger en pocas épocas si los datos son linealmente separables
# - Si no converge en 20 épocas, probablemente los datos no sean linealmente separables
#
# Nota: El entrenamiento puede terminar antes si el modelo clasifica todo correctamente
epocas = 20

print("="*60)
print("INICIANDO ENTRENAMIENTO")
print("="*60)
print(f"Tasa de aprendizaje: {tasa_aprendizaje}")
print(f"Épocas máximas: {epocas}")
print(f"Ejemplos de entrenamiento: {len(X)}\n")

###############################################################################
# ALGORITMO DE ENTRENAMIENTO DEL PERCEPTRÓN
###############################################################################
# 
# ¿Cómo aprende el perceptrón?
# Usa la "Regla de aprendizaje del perceptrón" propuesta por Rosenblatt (1958):
#
# Para cada ejemplo:
#   1. Hacer una predicción con los pesos actuales
#   2. Calcular el error: error = etiqueta_real - predicción
#   3. Si error ≠ 0, ajustar los pesos:
#      - Si error = 1 (predicción 0, real 1): aumentar pesos de features activos
#      - Si error = -1 (predicción 1, real 0): disminuir pesos de features activos
#
# ¿Por qué funciona?
# Matemáticamente, se demuestra que si los datos son linealmente separables,
# el perceptrón SIEMPRE converge a una solución (Teorema de Convergencia del Perceptrón).
#
###############################################################################

# Bucle de entrenamiento
for epoca in range(epocas):
    errores = 0
    
    # Recorremos cada ejemplo del dataset
    for i in range(len(X)):
        x_i = X[i]              # Vector de features de la frase i
        y_real = etiquetas[i]   # Etiqueta verdadera (0 o 1)
        y_pred = predecir(x_i)  # Predicción del modelo
        error = y_real - y_pred # Error: -1, 0, o 1
        
        # Solo ajustamos pesos si hubo error
        if error != 0:
            # Actualización de pesos:
            # Si error = 1 (falso negativo): aumentamos pesos donde x_i = 1
            # Si error = -1 (falso positivo): disminuimos pesos donde x_i = 1
            #
            # Ejemplo numérico:
            # Si x_i = [1, 0, 1], error = 1, tasa = 0.1
            # Entonces:
            # pesos += 0.1 × 1 × [1, 0, 1] = [0.1, 0, 0.1]
            # Solo los pesos de palabras presentes (x_i=1) cambian
            pesos += tasa_aprendizaje * error * x_i
            
            # Actualización del bias:
            # El bias no depende de ninguna entrada específica, solo del error
            bias += tasa_aprendizaje * error
            
            errores += 1
    
    print(f"Época {epoca + 1:2d}: Errores = {errores}")
    
    # Condición de convergencia:
    # Si errores = 0, el modelo clasificó correctamente TODOS los ejemplos
    # No tiene sentido seguir entrenando
    if errores == 0:
        print(f"\nConvergencia alcanzada en época {epoca + 1}")
        print("El modelo clasifica correctamente todos los ejemplos de entrenamiento.")
        break

print("\n" + "="*60)
print("ENTRENAMIENTO FINALIZADO")
print("="*60)

## 5. Análisis de los pesos aprendidos

Después del entrenamiento, los pesos nos indican qué tan importante es cada palabra para determinar el sentimiento.

- **Pesos positivos**: Palabras asociadas a sentimiento positivo
- **Pesos negativos**: Palabras asociadas a sentimiento negativo
- **Pesos cercanos a 0**: Palabras poco informativas

In [ ]:
print("Pesos aprendidos después del entrenamiento:\n")
print(f"{'Palabra':<15} {'Peso':>10} {'Interpretación'}")
print("-" * 50)

for palabra, peso in sorted(zip(vocabulario, pesos), key=lambda x: x[1], reverse=True):
    if peso > 0.1:
        interpretacion = "→ Positivo"
    elif peso < -0.1:
        interpretacion = "→ Negativo"
    else:
        interpretacion = "→ Neutral"
    
    print(f"{palabra:<15} {peso:>10.3f}  {interpretacion}")

print(f"\nBias final: {bias:.3f}")
print("\nEl bias representa el umbral base del modelo.")

## 6. Prueba con nuevas frases

Ahora vamos a ver cómo se comporta nuestro perceptrón con frases nuevas que no vio durante el entrenamiento.

Esta es la verdadera prueba: ¿puede generalizar lo que aprendió a casos nuevos?

In [ ]:
# Frases nuevas para testeo
frases_prueba = [
    "No aguanto este calor",
    "Qué hermoso día para pasear",
    "Detesto levantarme temprano"
]

print("="*60)
print("PREDICCIONES EN FRASES NUEVAS")
print("="*60)

# Vectorizamos las frases nuevas
X_prueba = np.array([vectorizar(frase, vocabulario) for frase in frases_prueba])
predicciones = [predecir(x) for x in X_prueba]

# Mostramos los resultados
for i, (frase, pred, vector) in enumerate(zip(frases_prueba, predicciones, X_prueba), 1):
    resultado = "Positivo" if pred == 1 else "Negativo"
    print(f"\nFrase {i}: '{frase}'")
    print(f"  Vector: {vector}")
    print(f"  Predicción: {resultado}")
    
    # Mostramos qué palabras del vocabulario detectó
    palabras_detectadas = [vocabulario[j] for j in range(len(vocabulario)) if vector[j] == 1]
    if palabras_detectadas:
        print(f"  Palabras clave detectadas: {', '.join(palabras_detectadas)}")
    else:
        print(f"  No se detectaron palabras del vocabulario")

print("\n" + "="*60)

## 7. Reflexión final

### ¿Qué aprendimos?

1. **Funcionamiento de una neurona artificial básica**: El perceptrón es el bloque fundamental de las redes neuronales. Aprendimos cómo combina entradas ponderadas y aplica una función de activación.

2. **Proceso de entrenamiento**: Vimos cómo un modelo aprende ajustando sus pesos iterativamente basándose en los errores que comete. Este principio se extiende a redes neuronales más complejas.

3. **Representación de texto**: Usamos bag-of-words, una técnica simple pero efectiva para convertir texto en números que las máquinas pueden procesar.

### Limitaciones observadas

1. **No considera el orden**: "No me gusta" vs "Me gusta, no" se representan igual
2. **Vocabulario limitado**: Solo conoce las palabras que definimos manualmente
3. **Modelo lineal**: Solo puede aprender patrones linealmente separables
4. **Sin contexto**: No entiende sarcasmo, ironía o matices del lenguaje
5. **Dataset pequeño**: Con solo 6 ejemplos, la generalización es limitada

### ¿Qué sigue?

En el próximo laboratorio, vamos a ver cómo las **redes neuronales multicapa** (MLP) pueden capturar patrones más complejos usando múltiples perceptrones organizados en capas. Esto nos va a permitir:

- Aprender representaciones no lineales
- Capturar interacciones entre features
- Mejorar la capacidad de generalización

Más adelante veremos cómo las **redes recurrentes** (LSTM) pueden procesar el orden de las palabras y, finalmente, cómo los **Transformers** revolucionaron el procesamiento de lenguaje natural.